In [52]:
import pandas as pd 
import numpy as np 

In [53]:
df = pd.read_csv("gurgaon_properties_post_feature_selection_with_ordinal_encoding.csv.").drop_duplicates()

In [64]:
df.head()

,property_type,sector,bedRoom,bathroom,balcony,floortype,agePossession,built_up_area,store room,servant room,furnishing_type,luxury_score,price
0,0.0,81.0,6.0,7.0,4.0,1.0,3.0,5333.00,0.0,1.0,0.0,2.0,5.00
1,0.0,45.0,2.0,2.0,3.0,2.0,1.0,625.00,0.0,0.0,0.0,1.0,0.60
2,0.0,90.0,3.0,4.0,2.0,0.0,1.0,2389.00,0.0,1.0,0.0,1.0,1.90
3,0.0,67.0,4.0,4.0,2.0,2.0,1.0,2567.00,0.0,1.0,0.0,1.0,6.75
4,0.0,56.0,3.0,4.0,3.0,2.0,0.0,1750.75,0.0,1.0,1.0,2.0,1.98


In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3507 entries, 0 to 3539
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   property_type    3507 non-null   float64
 1   sector           3507 non-null   float64
 2   bedRoom          3507 non-null   float64
 3   bathroom         3507 non-null   float64
 4   balcony          3507 non-null   float64
 5   floortype        3507 non-null   float64
 6   agePossession    3507 non-null   float64
 7   built_up_area    3507 non-null   float64
 8   store room       3507 non-null   float64
 9   servant room     3507 non-null   float64
 10  furnishing_type  3507 non-null   float64
 11  luxury_score     3507 non-null   float64
 12  price            3507 non-null   float64
dtypes: float64(13)
memory usage: 383.6 KB


In [56]:
X = df.drop(columns = ["price"])

In [57]:
y= df["price"]

In [58]:
X.head(2)

,property_type,sector,bedRoom,bathroom,balcony,floortype,agePossession,built_up_area,store room,servant room,furnishing_type,luxury_score
0,0.0,81.0,6.0,7.0,4.0,1.0,3.0,5333.0,0.0,1.0,0.0,2.0
1,0.0,45.0,2.0,2.0,3.0,2.0,1.0,625.0,0.0,0.0,0.0,1.0


In [59]:
y

0       5.00
1       0.60
2       1.90
3       6.75
4       1.98
        ... 
3535    3.55
3536    1.09
3537    0.40
3538    1.23
3539    2.85
Name: price, Length: 3507, dtype: float64

In [60]:
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error

In [61]:
# AS DISCUSSED EARLIER WE USE LOG TRANSFOMATION FOR OUT TARGET VARIABLE AS IT IS RIGHT SKEWED 

In [62]:
y = np.log1p(y)

In [66]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error

# Load the dataset
df = pd.read_csv("gurgaon_properties_post_feature_selection_with_ordinal_encoding.csv").drop_duplicates()

# Reset index properly
df.reset_index(drop=True, inplace=True)

# Check for null values
print(df.isnull().sum())

# Separate features and target variable
X = df.drop(columns=["price"])
y = df["price"]

# Log transform the target variable
y_transformed = np.log1p(y)

# Define columns to encode
columns_to_encode = ['sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_score', 'floortype']

# Create a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['property_type', 'bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), columns_to_encode)
    ],
    remainder='passthrough'
)

# Create a pipeline with SVR as the regressor
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', SVR(kernel='rbf'))
])

# Fit the pipeline on the entire dataset to ensure all categories are included
pipeline.fit(X, y_transformed)

# Perform K-fold cross-validation for R² score
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
r2_scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

# Calculate mean and standard deviation of R² scores
mean_r2 = r2_scores.mean()
std_r2 = r2_scores.std()

print(f"Mean R2 Score: {mean_r2}")
print(f"Standard Deviation of R2 Scores: {std_r2}")

# Perform K-fold cross-validation for MAE
mae_scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='neg_mean_absolute_error')

# Calculate mean and standard deviation of MAE scores
mean_mae = -mae_scores.mean()
std_mae = mae_scores.std()

print(f"Mean MAE Score: {mean_mae}")
print(f"Standard Deviation of MAE Scores: {std_mae}")

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_transformed, test_size=0.2, random_state=42)

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Predict on the test data
y_pred = pipeline.predict(X_test)

# Inverse transform to get actual prices
y_pred_actual = np.expm1(y_pred)
y_test_actual = np.expm1(y_test)

# Calculate mean absolute error
mae = mean_absolute_error(y_test_actual, y_pred_actual)
print(f"Mean Absolute Error: {mae}")



property_type      0
sector             0
bedRoom            0
bathroom           0
balcony            0
floortype          0
agePossession      0
built_up_area      0
store room         0
servant room       0
furnishing_type    0
luxury_score       0
price              0
dtype: int64


C:\Users\guriv\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Mean R2 Score: 0.8825412436331753
Standard Deviation of R2 Scores: 0.023424469928273784


C:\Users\guriv\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Mean MAE Score: 0.12832780576087688
Standard Deviation of MAE Scores: 0.0069544203400250144
Mean Absolute Error: 0.6100292314666849
